In [1]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 31.2 MB/s 


In [29]:
import warnings
warnings.filterwarnings("ignore")
import csv
import pandas as pd 
import numpy as np
import datetime 
import time 
import numpy as np 
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns 
from matplotlib import rcParams   
import math
import pickle
import os
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import networkx as nx
import pdb
import pickle
from tqdm.notebook import tqdm
import os
import random
from sklearn.model_selection import train_test_split
from pandas import read_hdf
from sklearn.metrics import f1_score

# Algo imports
import tensorflow as tf
from tensorflow import keras
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
data_path = "gdrive/My Drive/Twitch Link Prediction/DE/"

In [9]:
df_train = pd.read_csv(data_path+"de_pos_train_91.csv", index_col=0)
print(df_train.shape)
print(df_train.head())

(137671, 3)
         src  dest  link
72151   2716  4596     1
59689   2233  9423     1
78618   2916  3937     1
1040      53  1289     1
135261  6504  7787     1


In [11]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.DiGraph())
degrees = nx.degree(train_graph)
degrees = dict(degrees)

features = {}
for key, val in degrees.items():
  features[key] = [val]

pr = nx.pagerank(train_graph, alpha=0.85)
pr = dict(pr)
for key, val in pr.items():
  features[key].append(val)

katz = nx.katz_centrality(train_graph,alpha=0.005,beta=1)
katz = dict(katz)
for key, val in katz.items():
  features[key].append(val)

nx.set_node_attributes(train_graph, features, "features")

In [12]:
G = StellarGraph.from_networkx(train_graph, node_features="features")

In [13]:
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 9498, Edges: 137671

 Node types:
  default: [9498]
    Features: float32 vector, length 3
    Edge types: default-default->default

 Edge types:
    default-default->default: [137671]
        Weights: all 1 (default)
        Features: none


In [15]:
fullbatch_generator = FullBatchNodeGenerator(G, sparse=True)
gcn_model = GCN(layer_sizes=[64], activations=['relu'], generator=fullbatch_generator)

corrupted_generator = CorruptedGenerator(fullbatch_generator)
gen = corrupted_generator.flow(G.nodes())

Using GCN (local pooling) filters...


In [16]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

In [17]:
batch_size=100
epochs=50
es = EarlyStopping(monitor="loss", min_delta=0, patience=4, restore_best_weights=True,verbose=1)

In [18]:
history = model.fit(gen, epochs=epochs, verbose=1, callbacks=[es])

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 32.4387
Epoch 2/50
1/1 [==============================] - 0s 33ms/step - loss: 26.5561
Epoch 3/50
1/1 [==============================] - 0s 30ms/step - loss: 20.7218
Epoch 4/50
1/1 [==============================] - 0s 31ms/step - loss: 14.9096
Epoch 5/50
1/1 [==============================] - 0s 30ms/step - loss: 9.1268
Epoch 6/50
1/1 [==============================] - 0s 32ms/step - loss: 3.4312
Epoch 7/50
1/1 [==============================] - 0s 33ms/step - loss: 1.1214
Epoch 8/50
1/1 [==============================] - 0s 30ms/step - loss: 2.8281
Epoch 9/50
1/1 [==============================] - 0s 28ms/step - loss: 3.9060
Epoch 10/50
1/1 [==============================] - 0s 31ms/step - loss: 4.9599
Epoch 11/50
1/1 [==============================] - 0s 35ms/step - loss: 5.7919
Epoch 11: early stopping


In [19]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = Model(inputs=x_emb_in, outputs=x_out)

In [20]:
train_gen = fullbatch_generator.flow(G.nodes())
emb = emb_model.predict(train_gen)

In [21]:
emb.shape

(9498, 64)

In [22]:
emb_dict = {}
for i, node in enumerate(list(G.nodes())):
  emb_dict[node] = np.array(emb[i].reshape(1,64))

In [23]:
combined = pd.read_csv(data_path+"/de_train_91.csv", index_col = 0)
combined.head()

,src,dest,link
115788,4957,9057,1
140262,7055,8106,1
146109,7744,7802,1
115014,7737,1331,0
115249,2152,7581,0


In [24]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  train_emb.append(comb_emb[0])
train_emb = np.array(train_emb)

0it [00:00, ?it/s]

In [25]:
train_y = combined['link']
train_y.shape

(272671,)

In [26]:
combined_test = pd.read_csv(data_path+"/de_test_91.csv", index_col = 0)
combined_test.head()

,src,dest,link
34503,1285,8630,1
118484,8578,6482,0
44808,538,2754,0
131958,5061,7162,0
58422,2172,3741,1


In [27]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_test.iterrows()):
  comb_emb = (emb_dict[row['src']]) * (emb_dict[row['dest']])
  test_emb.append(comb_emb[0])

print(type(test_emb))
test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_test['link'] 

0it [00:00, ?it/s]

<class 'list'>
(30467, 64)


In [ ]:
#train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [31]:
#reading
df_final_train = read_hdf(data_path + 'de_fea_sample/de_storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf(data_path + 'de_fea_sample/de_storage_sample_stage4.h5', 'test_df',mode='r')

In [32]:
y_train = df_final_train.link
y_test = df_final_test.link

In [33]:
df_final_train.drop(['src', 'dest','link'],axis=1,inplace=True)
df_final_test.drop(['src', 'dest','link'],axis=1,inplace=True)

In [34]:
# GCN only 
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7867114057595778 test Score 0.7767630820514602


In [35]:
# GCN only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7912491829751239 test Score 0.7768189976529063


In [36]:
# GCN only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(train_emb, train_y)
train_sc = f1_score(train_y,clf.predict(train_emb))
test_sc = f1_score(test_y,clf.predict(test_emb))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.7778284213372475 test Score 0.7727225475588204


In [37]:
# Heuristic only
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9534085120262697 test Score 0.8818799389990296


In [38]:
# Heuristic only
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.950799409621998 test Score 0.8858754758020663


In [39]:
# Heuristic only
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9524353621914599 test Score 0.8831285378986607


In [40]:
df_final_train = np.hstack((df_final_train, train_emb))
print(df_final_train.shape)
df_final_test = np.hstack((df_final_test, test_emb))
print(df_final_test.shape)

(272671, 120)
(30467, 120)


In [41]:
print(type(train_y), type(y_train))
for i in range(len(y_train.values)):
  if train_y.values[i] != y_train.values[i]: print(i)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [42]:
# Heuristic + GCN
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9535986514218705 test Score 0.8815698238801831


In [43]:
# Heuristic + GCN
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9493543816160936 test Score 0.8879732513762707


In [44]:
# Heuristic + GCN
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
clf.fit(df_final_train,y_train)
train_sc = f1_score(y_train,clf.predict(df_final_train))
test_sc = f1_score(y_test,clf.predict(df_final_test))
print('Train Score',train_sc,'test Score',test_sc)

Train Score 0.9527471964020187 test Score 0.8835971521393515
